# O PROBLEMA

Imagine agora que você foi contratado(a) como Expert em Data Analytics por um grande hospital para entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença.

Apesar de ser contratado(a) agora, a sua área observou que a utilização do estudo do PNAD-COVID 19 do IBGE seria uma ótima base para termos boas respostas ao problema proposto, pois são dados confiáveis.Porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas.

É sempre bom ressaltar que há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

## PNAD-COVID-19 do IBGE

O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

- a. Utilização de no máximo 20 questionamentos realizados na pesquisa;
- b. Utilizar 3 meses para construção da solução;
- c. Caracterização dos sintomas clínicos da população;
- d. Comportamento da população na época da COVID-19;
- e. Características econômicas da Sociedade;

Seu objetivo será trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19.

- Dica: Leiam com atenção a base de dados e toda a documentação que o site o PNAD-COVID-19 traz, principalmente os dicionários, que ajudam e muito no entendimento da Base de Dados.
- Dica 2: Utilizem o que já foi ensinado e consolidado nas outras fases para apresentar a resolução do projeto.

Lembre-se de que você poderá apresentar o desenvolvimento do seu projeto durante as lives com docentes. Essa é uma boa oportunidade para discutir sobre as dificuldades encontradas e pegar dicas valiosas com especialistas e colegas de turma.

Link para a base: https://covid19.ibge.gov.br/pnad-covid/

Outras fontes:

https://www.gov.br/saude/pt-br/assuntos/noticias/2021/janeiro/entenda-a-ordem-de-vacinacao-contra-a-covid-19-entre-os-grupos-prioritarios

https://www.gov.br/saude/pt-br/assuntos/coronavirus/vacinas/esquemas-vacinais

### Importando bibliotecas

In [2]:
import warnings 
warnings.filterwarnings(action = 'ignore')

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from pyspark.sql import SparkSession
import findspark
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json

### Sobre o banco de dados

O banco de dados foi desenvolvido no MySQL com o objetivo de centralizar as informações disponíveis na pesquisa PNAD-COVID-19 do IBGE. Optou-se por incluir todos os dados da pesquisa no banco de dados e, desses, extrair para análise somente os dados referentes aos meses de setembro, outubro e novembro de 2020. O esquema foi desenhado de modo que fosse permitido a inclusão de novos dados, para o caso em que haja continuação da pesquisa. 

#### Estrutura do banco de dados

O banco de dados é composto por uma tabela principal (dados_covid) que armazena todos os dados da pesquisa e tabelas auxiliares, que guardam as respostas obtidas para os questionamentos da pesquisa. As tabelas auxiliares abaixo guardam multiplos relacionamentos, pois muitos dos questionamentos da pesquisa têm como resposta um mesmo conjunto de opções.
    
- depara_respostas: guarda o conjunto de respostas "Sim", "Não", "Não sabe", “Ignorado” e "Não aplicável".
- depara_resultado_covid: guarda o conjunto de respostas "Positivo", "Negativo", "Inconclusivo", "Ainda não recebeu o resultado" e "Ignorado".

A tabela dados_covid possui chaves estrangeiras que relacionam os campos que possuem um mesmo conjunto de respostas às tabelas auxiliares acima. Houve a inserção de índices únicos (UNIQUE KEYS) nas tabelas para evitar duplicidade de dados, permitindo assim a inserção de novos dados. Cada linha da tabela dados_covid representa o conjunto de respostas de um entrevistado.

#### Estrutura das tabelas

dados_covid - armazena os dados da pesquisa:

- id - chave primária;
- ano – ano da pesquisa;
- uf - sigla da unidade da federação;
- ...demais campos da pesquisa.

depara_respostas - armazena as respostas padronizadas:

- id - chave primária;
- respostas_id - valores;
- name - "Sim", "Não", "Não sabe", "Ignorado", "Não aplicável".

depara_resultado_covid - armazena os sintomas investigados:

- id - chave primária;
- respostas_id – valores;
- name - "Positivo", "Negativo", "Inconclusivo", "Ainda não recebeu o resultado" e "Ignorado".

#### View

Uma view (pnad_covid_view) foi criada com o objetivo de facilitar consultas aos dados. Essa traz somente os dados os meses de setembro, outubro e novembro de campos consideradas mais relevantes para a análise, assim é possível manipular os dados de forma ágil, sem a necessidade de consultar a estrutura integral do banco.

Em síntese, o banco de dados foi modelado visando integridade e flexibilidade para as consultas. As tabelas auxiliares e a view agregam valor ao permitir padronização e consulta orientada para análise. A estrutura foi pensada para receber dados de pesquisas futuras, garantindo a continuidade do uso do banco.

### Conectando no MySQL

In [4]:
'''def mysql_connection(host, user, passwd, database=None):
    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')
    return engine.connect()

mysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')'''

"def mysql_connection(host, user, passwd, database=None):\n    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')\n    return engine.connect()\n\nmysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')"

In [5]:
'''statement = text('SELECT * FROM pnad_covid_view LIMIT 5')
response = mysql.execute(statement)
for row in response:
        print(row)'''

"statement = text('SELECT * FROM pnad_covid_view LIMIT 5')\nresponse = mysql.execute(statement)\nfor row in response:\n        print(row)"

### Criando sessão Spark

In [6]:
findspark.init()

In [7]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

23/11/04 00:43:43 WARN Utils: Your hostname, platero-Lenovo-IdeaPad-S145-15IWL resolves to a loopback address: 127.0.1.1; using 192.168.15.109 instead (on interface wlp2s0)
23/11/04 00:43:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/04 00:43:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/04 00:43:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
spark

### Importando dados da pesquisa

In [9]:
'''df_05 = spark.read.csv('dados/dados_importados/PNAD_COVID_052020.csv', sep=',', inferSchema=True, header=True)
df_06 = spark.read.csv('dados/dados_importados/PNAD_COVID_062020.csv', sep=',', inferSchema=True, header=True)
df_07 = spark.read.csv('dados/dados_importados/PNAD_COVID_072020.csv', sep=',', inferSchema=True, header=True)
df_08 = spark.read.csv('dados/dados_importados/PNAD_COVID_082020.csv', sep=',', inferSchema=True, header=True)
df_09 = spark.read.csv('dados/dados_importados/PNAD_COVID_092020.csv', sep=',', inferSchema=True, header=True)
df_10 = spark.read.csv('dados/dados_importados/PNAD_COVID_102020.csv', sep=',', inferSchema=True, header=True)
df_11 = spark.read.csv('dados/dados_importados/PNAD_COVID_112020.csv', sep=',', inferSchema=True, header=True)
'''

"df_05 = spark.read.csv('dados/dados_importados/PNAD_COVID_052020.csv', sep=',', inferSchema=True, header=True)\ndf_06 = spark.read.csv('dados/dados_importados/PNAD_COVID_062020.csv', sep=',', inferSchema=True, header=True)\ndf_07 = spark.read.csv('dados/dados_importados/PNAD_COVID_072020.csv', sep=',', inferSchema=True, header=True)\ndf_08 = spark.read.csv('dados/dados_importados/PNAD_COVID_082020.csv', sep=',', inferSchema=True, header=True)\ndf_09 = spark.read.csv('dados/dados_importados/PNAD_COVID_092020.csv', sep=',', inferSchema=True, header=True)\ndf_10 = spark.read.csv('dados/dados_importados/PNAD_COVID_102020.csv', sep=',', inferSchema=True, header=True)\ndf_11 = spark.read.csv('dados/dados_importados/PNAD_COVID_112020.csv', sep=',', inferSchema=True, header=True)\n"

### Tratando dados da pesquisa

In [10]:
#igualando número de colunas, pois a quantidade de questões divergem entre os meses da pesquisa
'''from pyspark.sql.functions import lit

def add_missing_cols(df):
    cols = df_11.columns
    new_cols = []

    for col_name in cols: #selecionando colunas faltantes
        if col_name not in df.columns:
            new_cols.append(col_name)

    for new_col_name in new_cols: #adicionando colunas faltantes
        df = df.withColumn(new_col_name, lit(None))

    return df.select(cols) #organizando colunas

df_05 = add_missing_cols(df_05)
df_06 = add_missing_cols(df_06)
df_07 = add_missing_cols(df_07)
df_08 = add_missing_cols(df_08)
df_09 = add_missing_cols(df_09)
df_10 = add_missing_cols(df_10)'''

'from pyspark.sql.functions import lit\n\ndef add_missing_cols(df):\n    cols = df_11.columns\n    new_cols = []\n\n    for col_name in cols: #selecionando colunas faltantes\n        if col_name not in df.columns:\n            new_cols.append(col_name)\n\n    for new_col_name in new_cols: #adicionando colunas faltantes\n        df = df.withColumn(new_col_name, lit(None))\n\n    return df.select(cols) #organizando colunas\n\ndf_05 = add_missing_cols(df_05)\ndf_06 = add_missing_cols(df_06)\ndf_07 = add_missing_cols(df_07)\ndf_08 = add_missing_cols(df_08)\ndf_09 = add_missing_cols(df_09)\ndf_10 = add_missing_cols(df_10)'

In [11]:
#concatenando DataFrames
'''
df = df_05.union(df_06)
df = df.union(df_07)
df = df.union(df_08)
df = df.union(df_09)
df = df.union(df_10)
df = df.union(df_11)'''

'\ndf = df_05.union(df_06)\ndf = df.union(df_07)\ndf = df.union(df_08)\ndf = df.union(df_09)\ndf = df.union(df_10)\ndf = df.union(df_11)'

In [12]:
'''df.select('C007C').where(df['V1013'] == '06').show(5)'''

"df.select('C007C').where(df['V1013'] == '06').show(5)"

### Exportando dados tratados da pesquisa para CSV

In [13]:
#exportando arquivo csv único
'''df.coalesce(1).write.csv(
    path='dados/dados_exportados',
    mode='overwrite',
    sep=',',
    header=True
)'''

"df.coalesce(1).write.csv(\n    path='dados/dados_exportados',\n    mode='overwrite',\n    sep=',',\n    header=True\n)"

### Exportando seguimentos dos dados tratados da pesquisa

A seguimentação torna processo de inserção dos dados no banco de dados mais ágil

In [14]:
#exportando seguimentos do DataFrame
'''n_splits = 5
n_rows = df.count() // n_splits
copy_df = df

for i in range(1, n_splits + 1):
    temp_df = copy_df.limit(n_rows)
    copy_df = copy_df.subtract(temp_df) #subtraindo conteúdo seguimentado
    
    #exportando seguimento
    temp_df.coalesce(1).write.csv(
    path=f'dados/dados_exportados/dados_seguimentados/seguimento_{i}',
    mode='overwrite',
    sep=',',
    header=True
    )'''

"n_splits = 5\nn_rows = df.count() // n_splits\ncopy_df = df\n\nfor i in range(1, n_splits + 1):\n    temp_df = copy_df.limit(n_rows)\n    copy_df = copy_df.subtract(temp_df) #subtraindo conteúdo seguimentado\n    \n    #exportando seguimento\n    temp_df.coalesce(1).write.csv(\n    path=f'dados/dados_exportados/dados_seguimentados/seguimento_{i}',\n    mode='overwrite',\n    sep=',',\n    header=True\n    )"

In [15]:
'''df_1 = spark.read.csv('dados/dados_exportados/dados_seguimentados/seguimento_1/*.csv', sep=',', inferSchema=True, header=True)
df_2 = spark.read.csv('dados/dados_exportados/dados_seguimentados/seguimento_2/*.csv', sep=',', inferSchema=True, header=True)'''

"df_1 = spark.read.csv('dados/dados_exportados/dados_seguimentados/seguimento_1/*.csv', sep=',', inferSchema=True, header=True)\ndf_2 = spark.read.csv('dados/dados_exportados/dados_seguimentados/seguimento_2/*.csv', sep=',', inferSchema=True, header=True)"

### Exportando dados sequimentados para o MySQL

In [16]:
'''
df = pd.read_csv('/home/platero/postech_techchallenge_fase_3/dados/dados_exportados/dados_seguimentados/seguimento_1.csv')
df.to_sql('dados_covid', mysql, if_exists='append', index=False)
mysql.execute(text('COMMIT'))

df = pd.read_csv('/home/platero/postech_techchallenge_fase_3/dados/dados_exportados/dados_seguimentados/seguimento_2.csv')
df.to_sql('dados_covid', mysql, if_exists='append', index=False)
mysql.execute(text('COMMIT'))

df = pd.read_csv('/home/platero/postech_techchallenge_fase_3/dados/dados_exportados/dados_seguimentados/seguimento_3.csv')
df.to_sql('dados_covid', mysql, if_exists='append', index=False)
mysql.execute(text('COMMIT'))

df = pd.read_csv('/home/platero/postech_techchallenge_fase_3/dados/dados_exportados/dados_seguimentados/seguimento_4.csv')
df.to_sql('dados_covid', mysql, if_exists='append', index=False)
mysql.execute(text('COMMIT'))

df = pd.read_csv('/home/platero/postech_techchallenge_fase_3/dados/dados_exportados/dados_seguimentados/seguimento_5.csv')
df.to_sql('dados_covid', mysql, if_exists='append', index=False)
mysql.execute(text('COMMIT'))
'''

"\ndf = pd.read_csv('/home/platero/postech_techchallenge_fase_3/dados/dados_exportados/dados_seguimentados/seguimento_1.csv')\ndf.to_sql('dados_covid', mysql, if_exists='append', index=False)\nmysql.execute(text('COMMIT'))\n\ndf = pd.read_csv('/home/platero/postech_techchallenge_fase_3/dados/dados_exportados/dados_seguimentados/seguimento_2.csv')\ndf.to_sql('dados_covid', mysql, if_exists='append', index=False)\nmysql.execute(text('COMMIT'))\n\ndf = pd.read_csv('/home/platero/postech_techchallenge_fase_3/dados/dados_exportados/dados_seguimentados/seguimento_3.csv')\ndf.to_sql('dados_covid', mysql, if_exists='append', index=False)\nmysql.execute(text('COMMIT'))\n\ndf = pd.read_csv('/home/platero/postech_techchallenge_fase_3/dados/dados_exportados/dados_seguimentados/seguimento_4.csv')\ndf.to_sql('dados_covid', mysql, if_exists='append', index=False)\nmysql.execute(text('COMMIT'))\n\ndf = pd.read_csv('/home/platero/postech_techchallenge_fase_3/dados/dados_exportados/dados_seguimentados/s

### Importando view do MySQL

In [17]:
'''res = mysql.execute(text('SELECT * FROM pnad_covid_view LIMIT 1149196'))
res = res.fetchall()
df = pd.DataFrame(res)'''

"res = mysql.execute(text('SELECT * FROM pnad_covid_view LIMIT 1149196'))\nres = res.fetchall()\ndf = pd.DataFrame(res)"

In [18]:
'''df.info()'''

'df.info()'

### Exportando view para arquivo CSV

Os dados serão analisados a partir de arquivo CSV dada limitação de hardware, haja visto que a base de dados no MySQL contempla todos os dados disponibilizados pela pesquisa PNAD-COVID 19 do IBGE

In [19]:
'''df.to_csv('dados/dados_exportados/2023-11-01_pnad_covid_view.csv', index=False)'''

"df.to_csv('dados/dados_exportados/2023-11-01_pnad_covid_view.csv', index=False)"

### Importando view do arquivo CSV

In [20]:
'''df = spark.read.csv('dados/dados_exportados/2023-11-01_pnad_covid_view.csv', sep=',', inferSchema=True, header=True)
df.show(5, truncate=False)'''

"df = spark.read.csv('dados/dados_exportados/2023-11-01_pnad_covid_view.csv', sep=',', inferSchema=True, header=True)\ndf.show(5, truncate=False)"

### Tratando view do arquivo CSV

#### Dicionários

In [21]:
'''estados = {
    'Acre': 'AC',
    'Alagoas': 'AL',
    'Amapá': 'AP',
    'Amazonas': 'AM',
    'Bahia': 'BA',
    'Ceará': 'CE', 
    'Distrito Federal': 'DF',
    'Espírito Santo': 'ES',
    'Goiás': 'GO',
    'Maranhão': 'MA',
    'Mato Grosso': 'MT',
    'Mato Grosso do Sul': 'MS',
    'Minas Gerais': 'MG',
    'Pará': 'PA',
    'Paraíba': 'PB',
    'Paraná': 'PR',
    'Pernambuco': 'PE',
    'Piauí': 'PI', 
    'Rio de Janeiro': 'RJ',
    'Rio Grande do Norte': 'RN',
    'Rio Grande do Sul': 'RS',
    'Rondônia': 'RO',
    'Roraima': 'RR',
    'Santa Catarina': 'SC',
    'São Paulo': 'SP',
    'Sergipe': 'SE',
    'Tocantins': 'TO'
}

regioes = {
    'AC': 'Norte',
    'AL': 'Nordeste',
    'AP': 'Norte',
    'AM': 'Norte',
    'BA': 'Nordeste',
    'CE': 'Nordeste',
    'DF': 'Centro-Oeste',
    'ES': 'Sudeste',
    'GO': 'Centro-Oeste',
    'MA': 'Nordeste',
    'MT': 'Centro-Oeste',
    'MS': 'Centro-Oeste', 
    'MG': 'Sudeste',
    'PA': 'Norte',
    'PB': 'Nordeste',
    'PR': 'Sul',
    'PE': 'Nordeste',
    'PI': 'Nordeste',
    'RJ': 'Sudeste',
    'RN': 'Nordeste',
    'RS': 'Sul',
    'RO': 'Norte',
    'RR': 'Norte',
    'SC': 'Sul',
    'SP': 'Sudeste',
    'SE': 'Nordeste',
    'TO': 'Norte'
}'''

"estados = {\n    'Acre': 'AC',\n    'Alagoas': 'AL',\n    'Amapá': 'AP',\n    'Amazonas': 'AM',\n    'Bahia': 'BA',\n    'Ceará': 'CE', \n    'Distrito Federal': 'DF',\n    'Espírito Santo': 'ES',\n    'Goiás': 'GO',\n    'Maranhão': 'MA',\n    'Mato Grosso': 'MT',\n    'Mato Grosso do Sul': 'MS',\n    'Minas Gerais': 'MG',\n    'Pará': 'PA',\n    'Paraíba': 'PB',\n    'Paraná': 'PR',\n    'Pernambuco': 'PE',\n    'Piauí': 'PI', \n    'Rio de Janeiro': 'RJ',\n    'Rio Grande do Norte': 'RN',\n    'Rio Grande do Sul': 'RS',\n    'Rondônia': 'RO',\n    'Roraima': 'RR',\n    'Santa Catarina': 'SC',\n    'São Paulo': 'SP',\n    'Sergipe': 'SE',\n    'Tocantins': 'TO'\n}\n\nregioes = {\n    'AC': 'Norte',\n    'AL': 'Nordeste',\n    'AP': 'Norte',\n    'AM': 'Norte',\n    'BA': 'Nordeste',\n    'CE': 'Nordeste',\n    'DF': 'Centro-Oeste',\n    'ES': 'Sudeste',\n    'GO': 'Centro-Oeste',\n    'MA': 'Nordeste',\n    'MT': 'Centro-Oeste',\n    'MS': 'Centro-Oeste', \n    'MG': 'Sudeste',\n   

#### Adicionando colunas

In [22]:
'''df = df.withColumn('sigla', df.uf).replace(to_replace=estados, subset=['sigla']) #adicionando sigla dos estados
df = df.withColumn('regiao', df.sigla).replace(to_replace=regioes, subset=['regiao']) #adicionando regioes dos estados
df.show(5, truncate=False)'''

"df = df.withColumn('sigla', df.uf).replace(to_replace=estados, subset=['sigla']) #adicionando sigla dos estados\ndf = df.withColumn('regiao', df.sigla).replace(to_replace=regioes, subset=['regiao']) #adicionando regioes dos estados\ndf.show(5, truncate=False)"

### Exportando dados tratados da view para arquivo CSV

In [23]:
#exportando arquivo csv único
'''df.coalesce(1).write.csv(
    path='dados/dados_exportados/dados_uteis',
    mode='overwrite',
    sep=',',
    header=True
)'''

"df.coalesce(1).write.csv(\n    path='dados/dados_exportados/dados_uteis',\n    mode='overwrite',\n    sep=',',\n    header=True\n)"

In [24]:
'''df.coalesce(1).write.csv(
    path='dados/dados_exportados/dados_uteis/dados_uteis_gz', 
    compression='gzip', 
    mode='overwrite',
    sep=',',
    header=True
)'''

"df.coalesce(1).write.csv(\n    path='dados/dados_exportados/dados_uteis/dados_uteis_gz', \n    compression='gzip', \n    mode='overwrite',\n    sep=',',\n    header=True\n)"

### Importanto dados tratados da view do arquivo CSV

In [25]:
df = spark.read.csv('dados/dados_exportados/dados_uteis/dados_uteis_gz/2023-11-01_pnad_covid_view.csv.gz', sep=',', inferSchema=True, header=True)
df.show(5, truncate=False)

23/11/04 00:44:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+--------------+------------------+-----+------+--------+------------------------------------+-----------------------------+-----------------------+-----------------------------+---------------------------------+--------------------+------------------------+------------------+---------------------------------+--------------------------+---------------------------+-------------------------------+----------------+-------------------------------+------------------------+---------------------+-----------------------+-----------------+-----------------------+--------------------+-------------+-----------------------+-----------+---------------------------+-------------+-----------------+---------------+-----+--------+
|data      |uf            |situacao_domicilio|idade|sexo  |cor_raca|escolaridade                        |questao_estabelecimento_saude|questao_permaneceu_casa|questao_remedio_conta_propria|questao_remedio_orientacao_medica|questao_hospital_SUS|questao_hospital_priva

### Análise exploratória das caracteristicas gerais

In [26]:
df_temp = df.createOrReplaceTempView('df_temp')

#### **<p style='color:gray'> Estatísticas descritivas da incidência da doença</p>**

**Qual é quantidade total de infectados?**

In [27]:
df_qtd_testes_positivos = spark.sql(
    '''
        SELECT count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
    '''
).toPandas()
df_qtd_testes_positivos.to_csv('dados/dados_streamlit/df_qtd_testes_positivos.csv', sep=',', index=False)
df_qtd_testes_positivos

,qtd_testes_positivos
0,32057


**Qual é a quantidade total de infectados sintomáticos?**

In [28]:
df_qtd_infectados_sintomaticos = spark.sql(
    '''
        SELECT count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND sintoma_covid = 'Sim'

    '''
).toPandas()
df_qtd_infectados_sintomaticos.to_csv('dados/dados_streamlit/df_qtd_infectados_sintomaticos.csv', sep=',', index=False)
df_qtd_infectados_sintomaticos

,qtd_testes_positivos
0,3889


**Qual é a quantidade total de infectados sintomáticos?**

In [29]:
df_qtd_infectados_assintomaticos = spark.sql(
    '''
        SELECT count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND sintoma_covid = 'Não'

    '''
).toPandas()
df_qtd_infectados_assintomaticos.to_csv('dados/dados_streamlit/df_qtd_infectados_assintomaticos.csv', sep=',', index=False)
df_qtd_infectados_assintomaticos

,qtd_testes_positivos
0,28168


**Qual é a quantidade total de infectados que foram internados?**

In [30]:
df_qtd_infectados_internados = spark.sql(
    '''
        SELECT count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND questao_internacao = 'Sim'

    '''
).toPandas()
df_qtd_infectados_assintomaticos.to_csv('dados/dados_streamlit/df_qtd_infectados_internados.csv', sep=',', index=False)
df_qtd_infectados_internados

,qtd_testes_positivos
0,222


**Qual é a quantidade total de infectados internados com respiração artificial?**

In [31]:
df_qtd_infectados_internados_respiracao_artificial = spark.sql(
    '''
        SELECT count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND questao_internacao = 'Sim'

    '''
).toPandas()
df_qtd_infectados_internados_respiracao_artificial.to_csv('dados/dados_streamlit/df_qtd_infectados_internados_respiracao_artificial.csv', sep=',', index=False)
df_qtd_infectados_internados_respiracao_artificial

,qtd_testes_positivos
0,222


**Qual é quantidade de infectados por sexo?**

In [32]:
df_qtd_testes_positivos_sexo = spark.sql(
    '''
        SELECT sexo, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY sexo
    '''
).toPandas()
fig = px.bar(
    data_frame=df_qtd_testes_positivos_sexo.sort_values('qtd_testes_positivos', ascending=False), 
    x = 'sexo', 
    y = 'qtd_testes_positivos',
    color='sexo',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos': 'Quantidade',
        'sexo': 'Sexo'
    }
)
fig.update_layout(
    title='<b>Quantidade de infectados por sexo</b>',
    legend_title='Legenda',
    width=600, 
    height=400
)
fig.show()
df_qtd_testes_positivos_sexo.to_csv('dados/dados_streamlit/df_qtd_testes_positivos_sexo.csv', sep=',', index=False)

**Qual é a taxa  de incidência por sexo?**

Verifica-se que houve mais contaminação no grupo de mulheres validamente testadas.

In [33]:
df_qtd_testes_validos_sexo = spark.sql(
    '''
        SELECT sexo, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY sexo
    '''
).toPandas()
df_qtd_testes_positivos_sexo = spark.sql(
    '''
        SELECT sexo, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY sexo
    '''
).toPandas()
df_taxa_incidencia_sexo = pd.merge(df_qtd_testes_validos_sexo, df_qtd_testes_positivos_sexo, on='sexo')
df_taxa_incidencia_sexo['taxa_incidencia_mil_habitantes'] = ((df_taxa_incidencia_sexo['qtd_testes_positivos'] / df_taxa_incidencia_sexo['qtd_testes_validos']) * 1000).round().astype(int)
fig = px.pie(
    data_frame=df_taxa_incidencia_sexo.sort_values('taxa_incidencia_mil_habitantes', ascending=False), 
    values = 'taxa_incidencia_mil_habitantes', 
    names = 'sexo',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'taxa_incidencia_mil_habitantes': 'Taxa de incidência por (mil habitantes)',
        'sexo': 'Sexo'
    }
)
fig.update_layout(
    title='<b>Diferença percentual da taxa de incidência por sexo</b>',
    legend_title='Legenda',
    width=600, 
    height=400
)
fig.show()
df_taxa_incidencia_sexo.to_csv('dados/dados_streamlit/df_taxa_incidencia_sexo.csv', sep=',', index=False)

**Qual é quantidade de infectados por cor/raça?**

In [34]:
df_qtd_testes_positivos_cor_raca = spark.sql(
    '''
        SELECT cor_raca, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND cor_raca != 'Ignorado'
        GROUP BY cor_raca
    '''
).toPandas()
fig = px.bar(
    data_frame=df_qtd_testes_positivos_cor_raca.sort_values('qtd_testes_positivos', ascending=False), 
    x = 'cor_raca', 
    y = 'qtd_testes_positivos',
    color='cor_raca',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos': 'Quantidade',
        'cor_raca': 'Cor/raça'
    }
)
fig.update_layout(
    title='<b>Quantidade de infectados por cor/raça</b>',
    legend_title='Legenda',
    width=600, 
    height=400
)
fig.show()
df_qtd_testes_positivos_cor_raca.to_csv('dados/dados_streamlit/df_qtd_testes_positivos_cor_raca.csv', sep=',', index=False)

**Qual é a taxa  de incidência por cor/raça?**

In [35]:
df_qtd_testes_validos_cor_raca = spark.sql(
    '''
        SELECT cor_raca, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo') AND cor_raca != 'Ignorado'
        GROUP BY cor_raca
    '''
).toPandas()
df_qtd_testes_positivos_cor_raca = spark.sql(
    '''
        SELECT cor_raca, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND cor_raca != 'Ignorado'
        GROUP BY cor_raca
    '''
).toPandas()
df_taxa_incidencia_cor_raca = pd.merge(df_qtd_testes_validos_cor_raca, df_qtd_testes_positivos_cor_raca, on='cor_raca')
df_taxa_incidencia_cor_raca['taxa_incidencia_mil_habitantes'] = ((df_taxa_incidencia_cor_raca['qtd_testes_positivos'] / df_taxa_incidencia_cor_raca['qtd_testes_validos']) * 1000).round().astype(int)
fig = px.pie(
    data_frame=df_taxa_incidencia_cor_raca.sort_values('taxa_incidencia_mil_habitantes', ascending=False), 
    values = 'taxa_incidencia_mil_habitantes', 
    names = 'cor_raca',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'taxa_incidencia_mil_habitantes': 'Taxa de incidência por (mil habitantes)',
        'cor_raca': 'Cor/raça'
    }
)
fig.update_layout(
    title='<b>Diferença percentual da taxa de incidência por cor/raça</b>',
    legend_title='Legenda',
    width=600, 
    height=400
)
fig.show()
df_taxa_incidencia_cor_raca.to_csv('dados/dados_streamlit/df_taxa_incidencia_cor_raca.csv', sep=',', index=False)

**Qual é quantidade de infectados por escolaridade?**

In [36]:
df_qtd_testes_positivos_escolaridade = spark.sql(
    '''
        SELECT escolaridade, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY escolaridade
    '''
).toPandas()
fig = px.bar(
    data_frame=df_qtd_testes_positivos_escolaridade.sort_values('qtd_testes_positivos', ascending=False), 
    x = 'escolaridade', 
    y = 'qtd_testes_positivos',
    color='escolaridade',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos': 'Quantidade',
        'escolaridade': 'Escolaridade'
    }
)
fig.update_layout(
    title='<b>Quantidade de infectados por escolaridade</b>',
    showlegend=False,
    width=800, 
    height=600
)
fig.update_xaxes(tickangle=45)
fig.show()
df_qtd_testes_positivos_escolaridade.to_csv('dados/dados_streamlit/df_qtd_testes_positivos_escolaridade.csv', sep=',', index=False)

**Qual é a taxa  de incidência por escolaridade?**

In [37]:
df_qtd_testes_validos_escolaridade = spark.sql(
    '''
        SELECT escolaridade, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY escolaridade
    '''
).toPandas()
df_qtd_testes_positivos_escolaridade = spark.sql(
    '''
        SELECT escolaridade, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' 
        GROUP BY escolaridade
    '''
).toPandas()
df_taxa_incidencia_escolaridade = pd.merge(df_qtd_testes_validos_escolaridade, df_qtd_testes_positivos_escolaridade, on='escolaridade')
df_taxa_incidencia_escolaridade['taxa_incidencia_mil_habitantes'] = ((df_taxa_incidencia_escolaridade['qtd_testes_positivos'] / df_taxa_incidencia_escolaridade['qtd_testes_validos']) * 1000).round().astype(int)
fig = px.bar(
    data_frame=df_taxa_incidencia_escolaridade.sort_values('taxa_incidencia_mil_habitantes', ascending=False),
    x='escolaridade',
    y='taxa_incidencia_mil_habitantes',
    color='escolaridade',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'taxa_incidencia_mil_habitantes': 'Taxa de incidência por (mil habitantes)',
        'escolaridade': 'Escolaridade'
    }
)
fig.update_layout(
    title='<b>Diferença percentual da taxa de incidência por escolaridade</b>',
    yaxis_title='%',
    showlegend=False,
    width=800,  
    height=600
)
fig.update_xaxes(tickangle=45)
fig.show()
df_taxa_incidencia_escolaridade.to_csv('dados/dados_streamlit/df_taxa_incidencia_escolaridade.csv', sep=',', index=False)

**Evolução**

In [38]:
df_qtd_evolucao_mensal_taxa_infectados = spark.sql(
    '''
        SELECT data, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY data
    '''
).toPandas()
fig = px.bar(
    data_frame=df_qtd_evolucao_mensal_taxa_infectados,
    x='data',
    y='qtd_testes_positivos',
    color='data',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos': 'Quantidade',
        'data': 'Data'
    }
)
fig.update_layout(
    title='<b>Evolução mensal da quantidade de infectados</b>',
    showlegend=False,
    width=600, 
    height=400
)
df_qtd_evolucao_mensal_taxa_infectados.to_csv('dados/dados_streamlit/df_qtd_evolucao_mensal_taxa_infectados.csv', sep=',', index=False)

#### **<p style='color:gray'> Estatísticas geoespaciais da incidência da doença</p>**

**Qual é a taxa de incidência nacional?**

In [39]:
df_qtd_testes_validos = spark.sql(
    '''
        SELECT count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
    '''
).toPandas()
df_qtd_testes_positivos = spark.sql(
    '''
        SELECT count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
    '''
).toPandas()
df_taxa_incidencia = df_qtd_testes_validos
df_taxa_incidencia['taxa_incidencia_mil_habitantes'] = ((df_qtd_testes_positivos['qtd_testes_positivos'] / df_qtd_testes_validos['qtd_testes_validos']) * 1000).round().astype(int)
df_taxa_incidencia['taxa_incidencia_mil_habitantes'].mean().astype(int)
df_taxa_incidencia.to_csv('dados/dados_streamlit/df_taxa_incidencia.csv', sep=',', index=False)

**Qual é quantidade de infectados por zona de domicílio?**

Não foi constatada diferença significativa na ataxa de incidência por região.

In [40]:
df_qtd_testes_positivos_zona = spark.sql(
    '''
        SELECT situacao_domicilio, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY situacao_domicilio
    '''
).toPandas()
fig = px.bar(
    data_frame=df_qtd_testes_positivos_zona.sort_values('qtd_testes_positivos', ascending=False),
    x='situacao_domicilio',
    y='qtd_testes_positivos',
    color='situacao_domicilio',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos': 'Quantidade',
        'situacao_domicilio': 'Zona de domicílio'
    }
)
fig.update_layout(
    title='<b>Quantidade de infectados por zona de domicílio</b>',
    legend_title='Legenda',
    width=600, 
    height=400
)
fig.show()
df_qtd_testes_positivos_zona.to_csv('dados/dados_streamlit/df_qtd_testes_positivos_zona.csv', sep=',', index=False)

**Qual é a diferença percentual da taxa de incidência por zona de domicílio?**

In [41]:
df_qtd_testes_validos_zona = spark.sql(
    '''
        SELECT situacao_domicilio, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY situacao_domicilio
    '''
).toPandas()
df_qtd_testes_positivos_zona = spark.sql(
    '''
        SELECT situacao_domicilio, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY situacao_domicilio
    '''
).toPandas()
df_taxa_incidencia_zona = pd.merge(df_qtd_testes_validos_zona, df_qtd_testes_positivos_zona, on='situacao_domicilio')
df_taxa_incidencia_zona['taxa_incidencia_mil_habitantes'] = ((df_taxa_incidencia_zona['qtd_testes_positivos'] / df_taxa_incidencia_zona['qtd_testes_validos']) * 1000).round().astype(int)
fig = px.pie(
    data_frame=df_taxa_incidencia_zona.sort_values('taxa_incidencia_mil_habitantes', ascending=False),
    values = 'taxa_incidencia_mil_habitantes', 
    names = 'situacao_domicilio',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'taxa_incidencia_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'situacao_domicilio': 'Zona de domicílio'
    }
)
fig.update_layout(
    title='<b>Diferença percentual da taxa de incidência por zona de domicílio</b>',
    legend_title='Legenda',
    width=600, 
    height=400
)
fig.show()
df_taxa_incidencia_zona.to_csv('dados/dados_streamlit/df_taxa_incidencia_zona.csv', sep=',', index=False)

**Qual é quantidade de infectados por região?**

In [42]:
df_qtd_testes_positivos_regiao = spark.sql(
    '''
        SELECT regiao, count(teste_covid) as qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY regiao
    '''
).toPandas()
fig = px.bar(
    data_frame=df_qtd_testes_positivos_regiao.sort_values('qtd_testes_positivos', ascending=False),
    x='regiao',
    y='qtd_testes_positivos',
    color='regiao',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos': 'Quantidade',
        'regiao': 'Região'
    }
)
fig.update_layout(
    title='<b>Quantidade de infectados por região</b>',
    showlegend=False,
    width=600, 
    height=400
)
fig.update_xaxes(tickangle=45)
fig.show()
df_qtd_testes_positivos_regiao.to_csv('dados/dados_streamlit/df_qtd_testes_positivos_regiao.csv', sep=',', index=False)

**Qual é a quantidade de infectados por estado?**

In [43]:
df_qtd_testes_positivos_estado = spark.sql(
    '''
        SELECT uf, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY uf
    '''
).toPandas()
fig = px.bar(
    data_frame=df_qtd_testes_positivos_estado.sort_values('qtd_testes_positivos', ascending=False),
    x='uf',
    y='qtd_testes_positivos',
    color='uf',
    color_discrete_sequence=['#67000d'],
    labels={
        'qtd_testes_positivos': 'Quantidade',
        'uf': 'Estado'
    }
)
fig.update_layout(
    title='<b>Quantidade de infectados por estado</b>',
    yaxis_title='Quantidade de infectados',
    showlegend=False,
    width=600, 
    height=400
)
fig.update_xaxes(tickangle=45)
fig.show()
df_qtd_testes_positivos_estado.to_csv('dados/dados_streamlit/df_qtd_testes_positivos_estado.csv', sep=',', index=False)

**Qual é a taxa de incidência por região?**

In [44]:
df_qtd_testes_validos_regiao = spark.sql(
    '''
        SELECT regiao, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY regiao
    '''
).toPandas()
df_qtd_testes_positivos_regiao = spark.sql(
    '''
        SELECT regiao, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY regiao
    '''
).toPandas()
df_taxa_incidencia_regiao = pd.merge(df_qtd_testes_validos_regiao, df_qtd_testes_positivos_regiao, on='regiao')
df_taxa_incidencia_regiao['taxa_incidencia_mil_habitantes'] = ((df_taxa_incidencia_regiao['qtd_testes_positivos'] / df_taxa_incidencia_regiao['qtd_testes_validos']) * 1000).round().astype(int)
fig = px.bar(
    data_frame=df_taxa_incidencia_regiao.sort_values('taxa_incidencia_mil_habitantes', ascending=False),
    x='regiao',
    y='taxa_incidencia_mil_habitantes',
    color='regiao',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'taxa_incidencia_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'regiao': 'Região'
    }
)
fig.update_layout(
    title='<b>Taxa de incidência por região</b>',
    showlegend=False,
    width=600, 
    height=400
)
fig.update_xaxes(tickangle=45)
fig.show()
df_taxa_incidencia_regiao.to_csv('dados/dados_streamlit/df_taxa_incidencia_regiao.csv', sep=',', index=False)

**Qual é a diferença percentual da taxa de incidência por região?**

In [45]:
df_qtd_testes_validos_regiao = spark.sql(
    '''
        SELECT regiao, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY regiao
    '''
).toPandas()
df_qtd_testes_positivos_regiao = spark.sql(
    '''
        SELECT regiao, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY regiao
    '''
).toPandas()
df_taxa_incidencia_regiao = pd.merge(df_qtd_testes_validos_regiao, df_qtd_testes_positivos_regiao, on='regiao')
df_taxa_incidencia_regiao['taxa_incidencia_mil_habitantes'] = ((df_taxa_incidencia_regiao['qtd_testes_positivos'] / df_taxa_incidencia_regiao['qtd_testes_validos']) * 1000).round().astype(int)
fig = px.pie(
    data_frame=df_taxa_incidencia_regiao.sort_values('taxa_incidencia_mil_habitantes', ascending=False),
    values = 'taxa_incidencia_mil_habitantes', 
    names = 'regiao',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'taxa_incidencia_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'regiao': 'Região'
    }
)
fig.update_layout(
    title='<b>Diferença percentual da taxa de incidência por região</b>',
    width=600, 
    height=400
)
fig.show()
df_taxa_incidencia_regiao.to_csv('dados/dados_streamlit/df_taxa_incidencia_regiao.csv', sep=',', index=False)

**Qual é a média da taxa de incidência nas regiões?**

In [46]:
df_taxa_incidencia_regiao['taxa_incidencia_mil_habitantes'].mean().astype(int)

258

**Qual é a taxa de incidência por estado?**

Observa-se que a taxa é tanto maior quanto mais ao norte do país

In [47]:
df_qtd_testes_validos_estado = spark.sql(
    '''
        SELECT uf, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY uf
    '''
).toPandas()
df_qtd_testes_positivos_estado = spark.sql(
    '''
        SELECT uf, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY uf
    '''
).toPandas()
df_taxa_incidencia_estado = pd.merge(df_qtd_testes_validos_estado, df_qtd_testes_positivos_estado, on='uf')
df_taxa_incidencia_estado['taxa_incidencia_mil_habitantes'] = ((df_taxa_incidencia_estado['qtd_testes_positivos'] / df_taxa_incidencia_estado['qtd_testes_validos']) * 1000).round().astype(int)
fig = px.bar(
    data_frame=df_taxa_incidencia_estado.sort_values('taxa_incidencia_mil_habitantes', ascending=False),
    x='uf',
    y='taxa_incidencia_mil_habitantes',
    color='uf',
    color_discrete_sequence=['#67000d'],
    labels={
        'taxa_incidencia_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'uf': 'Estado'
    }
)
fig.update_layout(
    title='<b>Taxa de incidência por estado</b>',
    showlegend=False,
    width=800, 
    height=600
)
fig.update_xaxes(tickangle=45)
fig.show()
df_taxa_incidencia_estado.to_csv('dados/dados_streamlit/df_taxa_incidencia_estado.csv', sep=',', index=False)

**Qual é a média da taxa de incidência nos estados?**

In [48]:
df_taxa_incidencia_estado['taxa_incidencia_mil_habitantes'].mean().astype(int)

276

**Mapeamento de risco de acordo com a taxa de incidência**

geojson = json.load(open('dados/dados_streamlit/brasil_estados.json'))
df_qtd_testes_validos_estado = spark.sql(
    '''
        SELECT uf, sigla, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY uf, sigla
    '''
).toPandas()
df_qtd_testes_positivos_estado = spark.sql(
    '''
        SELECT sigla, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY sigla
    '''
).toPandas()
df_taxa_incidencia_estado = pd.merge(df_qtd_testes_validos_estado, df_qtd_testes_positivos_estado, on='sigla')
df_taxa_incidencia_estado['taxa_incidencia_mil_habitantes'] = ((df_taxa_incidencia_estado['qtd_testes_positivos'] / df_taxa_incidencia_estado['qtd_testes_validos']) * 1000).round().astype(int)
fig = px.choropleth(
    data_frame=df_taxa_incidencia_estado,
    geojson=geojson,
    locations='sigla',
    color='taxa_incidencia_mil_habitantes',
    hover_name='sigla',
    scope='south america',
    color_continuous_scale=px.colors.sequential.Reds,
    labels={
        'taxa_incidencia_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'sigla': 'Estado'
    }
)
fig.add_scattergeo(
    geojson=geojson,
    locations=df_taxa_incidencia_estado['sigla'],
    text=df_taxa_incidencia_estado['sigla'],
    textposition='middle center',
    mode='text',
    textfont=dict(
        size=14,
        color='black',
        
    )
)
fig.update_layout(
    title='<b>Mapa de risco de acordo com a taxa de incidência nos estados</b>',
    autosize=False,
    margin = dict(
        l=0,
        r=0,
        b=0,
        autoexpand=True
    ),
    width=800, 
    height=600
)
fig.show()
df_taxa_incidencia_estado.to_csv('dados/dados_streamlit/df_taxa_incidencia_estado.csv', sep=',', index=False)

#### **<p style='color:gray'> Estatísticas descritivas da incidência da doença em relação à faixa etária do esquema vacinal</p>**

**Qual é a taxa de incidência por faixa etária do esquema vacinal nos estados?**

O planejamento da vacinação para 2023 tem como proposta de esquema o uso de vacinas bivalentes com cepas atualizadas com dose de reforço (booster) para grupos específicos em maior vulnerabilidade, riscos para complicação e óbito e maior exposição, e o uso de vacinas monovalentes para dar início ou completar esquema vacinal das pessoas que não fazem parte dos grupos prioritários elegíveis para vacinação bivalente. Vale ressaltar que ambas as vacinas, monovalentes e bivalentes, agem do mesmo modo no organismo, estimulando o sistema imunológico a produzir anticorpos protetores e células de defesa contra o vírus SARS-CoV-2.

Quando infectada pelo vírus, a pessoa vacinada conseguirá combatê-lo rapidamente, pois já tem imunidade contra ele e assim evoluirá de modo assintomático ou com doença leve (maioria dos casos). O que difere a vacina monovalente da vacina bivalente é a capacidade das vacinas bivalentes de estimularem uma resposta imune mais efetiva contra a variante Ômicron, garantindo assim maior proteção contra a infecção pelo SARS-CoV-2. Cabe ressaltar, que as vacinas monovalentes continuam a proteger as pessoas completamente vacinadas contra hospitalizações, SRAG e óbito. As recomendações atuais para imunização contra a covid-19 no país são estabelecidas de acordo com as faixas etárias, os imunizantes disponíveis, as recomendações dos fabricantes e os resultados de estudos nacionais e internacionais. 

6 meses a 4 anos, 11 meses e 29 dias:

Para crianças que iniciaram o esquema vacinal recomendado para a faixa etária de 6 meses a 4 anos, 11 meses e 29 dias com a vacina Pfizer (frasco de tampa vinho) e finalizarão seu esquema após completar 5 anos de idade, essas deverão adotar o esquema recomendado para a faixa etária de 5 a 11 anos utilizando a vacina Pfizer (frasco de tampa laranja) para completar o seu esquema.


3 e 4 anos, 11 meses e 29 dias:

Para crianças de 3 e 4 anos, 11 meses e 29 dias que iniciaram o esquema vacinal com CoronaVac, o esquema primário é composto por duas doses (1ª Dose + 2ª Dose), com intervalo de quatro semanas entre a primeira e segunda doses. Nessa faixa etária há a indicação de uma dose de reforço preferencialmente com a vacina da Pfizer (frasco de tampa vinho), com intervalo de quatro meses após a segunda dose recebida. Na indisponibilidade dessa o reforço poderá ser realizado com a vacina CoronaVac. O registro da dose de reforço com a vacina CoronaVac no sistema de informação deverá ser 1º reforço (R1) e com a vacina Pfizer 3ª dose (D3). 

5 a 11 anos de idade:

Para crianças de 5 a 11 anos de idade, o esquema primário recomendado é composto por duas doses da vacina Covid-19 (1ª Dose + 2ª Dose). Para aquelas que iniciaram o esquema com o imunizante CoronaVac, o intervalo entre as doses é de quatro semanas, e, para as crianças que iniciaram o esquema com o imunizante Covid-19 Pfizer (frasco de tampa laranja), o intervalo é de oito semanas após a 1ª dose. A dose de reforço deve ser feita com intervalo mínimo de quatro meses após a segunda dose.

12 a 39 anos de idade:

Para as pessoas de 12 a 39 anos de idade não incluídas no grupo prioritário recomendado para receber as vacinas bivalentes e que não iniciaram a vacinação ou que estão com esquema vacinal incompleto, deverá ser realizado o esquema primário utilizando duas doses das vacinas Covid-19 (monovalente) e o reforço com intervalo mínimo de 4 meses entre as doses.

40 a 59 anos de idade:

Para adultos de 40 a 59 anos de idade não incluídos no grupo prioritário recomendado para receber as vacinas bivalentes, o esquema vacinal é composto por duas doses (1ª Dose + 2ª Dose) e duas doses de reforço (1º Reforço + 2º Reforço).

In [49]:
df_qtd_testes_validos_faixa_etaria_esquema_vacinal = spark.sql(
    '''
        SELECT
                (CASE
                    WHEN idade BETWEEN  1 AND 4 THEN '6 meses a 4 anos de idade'
                    WHEN idade BETWEEN 3 AND 4 THEN '3 e 4 anos de idade'
                    WHEN idade BETWEEN 5 AND 11 THEN '5 a 11 anos de idade'
                    WHEN idade BETWEEN 12 AND 39 THEN '12 a 39 anos de idade'
                    WHEN idade BETWEEN 40 AND 59 THEN '40 a 59 anos de idade'
                    WHEN idade >= 60 THEN 'Mais de 60 anos de idade'
                END) AS faixa_etaria,
                count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY faixa_etaria
    '''
).toPandas()
df_qtd_testes_positivos_faixa_etaria_esquema_vacinal = spark.sql(
    '''
        SELECT
                (CASE
                    WHEN idade BETWEEN  1 AND 4 THEN '6 meses a 4 anos de idade'
                    WHEN idade BETWEEN 3 AND 4 THEN '3 e 4 anos de idade'
                    WHEN idade BETWEEN 5 AND 11 THEN '5 a 11 anos de idade'
                    WHEN idade BETWEEN 12 AND 39 THEN '12 a 39 anos de idade'
                    WHEN idade BETWEEN 40 AND 59 THEN '40 a 59 anos de idade'
                    WHEN idade >= 60 THEN 'Mais de 60 anos de idade'
                END) AS faixa_etaria,
                count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY faixa_etaria
    '''
).toPandas()
df_taxa_incidencia_faixa_etaria_esquema_vacinal = pd.merge(df_qtd_testes_validos_faixa_etaria_esquema_vacinal, df_qtd_testes_positivos_faixa_etaria_esquema_vacinal, on='faixa_etaria')
df_taxa_incidencia_faixa_etaria_esquema_vacinal ['taxa_de_contagio_mil_habitantes']= ((df_taxa_incidencia_faixa_etaria_esquema_vacinal['qtd_testes_positivos'] / df_taxa_incidencia_faixa_etaria_esquema_vacinal['qtd_testes_validos']) * 1000).round().astype(int)
fig = px.bar(
    data_frame=df_taxa_incidencia_faixa_etaria_esquema_vacinal.sort_values('taxa_de_contagio_mil_habitantes', ascending=False),
    x='faixa_etaria',
    y='taxa_de_contagio_mil_habitantes',
    color='faixa_etaria',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'taxa_de_contagio_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'faixa_etaria': 'Faixa etária'
    }
)
fig.update_layout(
    title='<b>Taxa de incidência por faixa etária do esquema vacinal</b>',
    showlegend=False,
    width=800, 
    height=600
)
fig.update_xaxes(tickangle=45)
fig.show()
df_taxa_incidencia_faixa_etaria_esquema_vacinal.to_csv('dados/dados_streamlit/df_taxa_incidencia_faixa_etaria_esquema_vacinal.csv', sep=',', index=False)

### Análise exploratória das caracteristicas econômicas

#### **<p style='color:gray'> Estatísticas descritivas da incidência da doença em relação ao rendimento</p>**

**Qual é a quantidade de infectados por faixa de rendimento?**

In [50]:
df_qtd_testes_positivos_faixa_rendimento = spark.sql(
    '''
        SELECT faixa_rendimento, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY faixa_rendimento
    '''
).toPandas()
fig = px.bar(
    data_frame=df_qtd_testes_positivos_faixa_rendimento.sort_values('qtd_testes_positivos', ascending=False),
    x='faixa_rendimento',
    y='qtd_testes_positivos', 
    color='faixa_rendimento',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos': 'Quantidade',
        'faixa_rendimento': 'Faixa de rendimento (R$)'
    }
)
fig.update_layout(
    title='<b>Quantidade de infectados por faixa de rendimento</b>',
    showlegend=False,
    width=800, 
    height=600
)
fig.update_xaxes(tickangle=45)
fig.show()
df_qtd_testes_positivos_faixa_rendimento.to_csv('dados/dados_streamlit/df_qtd_testes_positivos_faixa_rendimento.csv', sep=',', index=False)

In [154]:
df_qtd_testes_positivos_regiao_estado_faixa_rendimento = spark.sql(
    '''
        SELECT regiao, uf, faixa_rendimento, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND faixa_rendimento != 'None'
        GROUP BY regiao, uf, faixa_rendimento
        ORDER BY regiao, qtd_testes_positivos DESC
    '''
).toPandas()
fig = px.treemap(
    df_qtd_testes_positivos_regiao_estado_faixa_rendimento, 
    path = ['regiao', 'uf', 'faixa_rendimento'], 
    values = 'qtd_testes_positivos', 
    color='qtd_testes_positivos',
    color_continuous_scale=px.colors.sequential.Reds
)
fig.update_layout(
    title='<b>Quantidade de infectados por faixa de rendimento</b>',
    legend_title='Legenda',
    width=800,  
    height=600
)
fig.show()
df_qtd_testes_positivos_regiao_estado_faixa_rendimento.to_csv('dados/dados_streamlit/df_qtd_testes_positivos_regiao_estado_faixa_rendimento.csv', sep=',', index=False)

**Qual é o percentual de infectados por faixa de rendimento?**

In [51]:
df_qtd_testes_validos_faixa_rendimento = spark.sql(
    '''
        SELECT faixa_rendimento, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo') AND faixa_rendimento != 'None'
        GROUP BY faixa_rendimento
    '''
).toPandas()
df_qtd_testes_positivos_faixa_rendimento = spark.sql(
    '''
        SELECT faixa_rendimento, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND faixa_rendimento != 'None'
        GROUP BY faixa_rendimento
    '''
).toPandas()
df_taxa_incidencia_faixa_rendimento = pd.merge(df_qtd_testes_validos_faixa_rendimento, df_qtd_testes_positivos_faixa_rendimento, on='faixa_rendimento')
df_taxa_incidencia_faixa_rendimento['taxa_incidencia_mil_habitantes'] = ((df_taxa_incidencia_faixa_rendimento['qtd_testes_positivos'] / df_taxa_incidencia_faixa_rendimento['qtd_testes_validos']) * 1000).round().astype(int)
fig = px.bar(
    data_frame=df_taxa_incidencia_faixa_rendimento.sort_values('taxa_incidencia_mil_habitantes', ascending=False),
    x='faixa_rendimento',
    y='taxa_incidencia_mil_habitantes', 
    color='faixa_rendimento',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'taxa_incidencia_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'faixa_rendimento': 'Faixa de rendimento (R$)'
    }
)
fig.update_layout(
    title='<b>Taxa de incidência por faixa de rendimento</b>',
    showlegend=False,
    width=800,  
    height=600
)
fig.show()
df_taxa_incidencia_faixa_rendimento.to_csv('dados/dados_streamlit/df_taxa_incidencia_faixa_rendimento.csv', sep=',', index=False)

In [52]:
df_testes_positivos_valor_medio_auxilio_emergencial = spark.sql(
    '''
        SELECT faixa_rendimento, sum(auxlio_emergencia_covid) as soma_auxlio_covid_faixa_rendimento, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' AND auxlio_emergencia_covid != 'Não aplicável' AND auxlio_emergencia_covid != 'Não' AND faixa_rendimento != 'None'
        GROUP BY faixa_rendimento
    '''
).toPandas()
df_testes_positivos_valor_medio_auxilio_emergencial['media_auxlio_covid_faixa_rendimento'] = (df_testes_positivos_valor_medio_auxilio_emergencial.soma_auxlio_covid_faixa_rendimento / df_testes_positivos_valor_medio_auxilio_emergencial.qtd_testes_positivos).round(decimals=2)
fig = px.bar(
    data_frame=df_testes_positivos_valor_medio_auxilio_emergencial.sort_values('media_auxlio_covid_faixa_rendimento', ascending=False),
    x='faixa_rendimento',
    y='media_auxlio_covid_faixa_rendimento', 
    color='faixa_rendimento',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'media_auxlio_covid_faixa_rendimento': 'Valor (R$)',
        'faixa_rendimento': 'Faixa de rendimento (R$)'
    }
)
fig.update_layout(
    title='<b>Valor médio do auxílio emergencial recebido entre infectados por faixa de rendimento</b>',
    showlegend=False,
    width=800, 
    height=600
)
fig.update_xaxes(tickangle=45)
fig.show()
df_testes_positivos_valor_medio_auxilio_emergencial.to_csv('dados/dados_streamlit/df_testes_positivos_valor_medio_auxilio_emergencial.csv', sep=',', index=False)

#### **<p style='color:gray'> Estatísticas descritivas de incidência da doença em relação as condições de trabalho</p>**

**Qual é a quantidade de infectados por tipo de trabalho?**

In [53]:
df_qtd_testes_positivos_tipo_trabalho = spark.sql(
    '''
        SELECT questao_tipo_trabalho_realizado, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND questao_tipo_trabalho_realizado != 'Não aplicável' AND questao_tipo_trabalho_realizado != 'Outros'
        GROUP BY questao_tipo_trabalho_realizado
    '''
).toPandas()
fig = px.bar(
    data_frame=df_qtd_testes_positivos_tipo_trabalho.sort_values('qtd_testes_positivos', ascending=False),
    x='qtd_testes_positivos',
    y='questao_tipo_trabalho_realizado', 
    color='questao_tipo_trabalho_realizado',
    color_discrete_sequence=['#67000d'],
    labels={
        'qtd_testes_positivos':'Quantidade',
        'questao_tipo_trabalho_realizado':'Tipo de trabalho realizado'
    }
)
fig.update_layout(
    title='<b>Quantidade de infectados por tipo de trabalho realizado</b>',
    showlegend=False,
    width=1200, 
    height=800
)
fig.show()
df_qtd_testes_positivos_tipo_trabalho.to_csv('dados/dados_streamlit/df_qtd_testes_positivos_tipo_trabalho.csv', sep=',', index=False)

**Qual é a percentual de infectados por tipo de trabalho?**

In [54]:
df_qtd_testes_validos_tipo_trabalho = spark.sql(
    '''
        SELECT questao_tipo_trabalho_realizado, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo') AND questao_tipo_trabalho_realizado != 'Não aplicável' AND questao_tipo_trabalho_realizado != 'Outros'
        GROUP BY questao_tipo_trabalho_realizado
    '''
).toPandas()
df_qtd_testes_positivos_tipo_trabalho = spark.sql(
    '''
        SELECT questao_tipo_trabalho_realizado, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND questao_tipo_trabalho_realizado != 'Não aplicável' AND questao_tipo_trabalho_realizado != 'Outros'
        GROUP BY questao_tipo_trabalho_realizado
    '''
).toPandas()
df_taxa_incidencia_tipo_trabalho = pd.merge(df_qtd_testes_validos_tipo_trabalho, df_qtd_testes_positivos_tipo_trabalho, on='questao_tipo_trabalho_realizado')
df_taxa_incidencia_tipo_trabalho['taxa_incidencia_mil_habitantes'] = ((df_taxa_incidencia_tipo_trabalho['qtd_testes_positivos'] / df_taxa_incidencia_tipo_trabalho['qtd_testes_validos']) * 1000).round().astype(int)
fig = px.bar(
    data_frame=df_taxa_incidencia_tipo_trabalho.sort_values('taxa_incidencia_mil_habitantes', ascending=False),
    x='taxa_incidencia_mil_habitantes',
    y='questao_tipo_trabalho_realizado', 
    color='questao_tipo_trabalho_realizado',
    color_discrete_sequence=['#67000d'],
    labels={
        'taxa_incidencia_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'questao_tipo_trabalho_realizado': 'Tipo de trabalho realizado'
    }
)
fig.update_layout(
    title='<b>Taxa de incidência por tipo de trabalho</b>',
    showlegend=False,
    width=1200, 
    height=800
)
fig.show()
df_taxa_incidencia_tipo_trabalho.to_csv('dados/dados_streamlit/df_taxa_incidencia_tipo_trabalho.csv', sep=',', index=False)

**Qual é o percentual de infectados por motivo de afastamento do trabalho?**

In [55]:
df_percentual_motivo_afastamento = spark.sql(
    '''
        SELECT questao_motivo_afastamento, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' AND questao_motivo_afastamento != 'Não aplicável'
        GROUP BY questao_motivo_afastamento
    '''
).toPandas()
df_percentual_motivo_afastamento['percentual_motivo_afastamento'] = ((df_percentual_motivo_afastamento.qtd_testes_positivos / sum(df_percentual_motivo_afastamento.qtd_testes_positivos)) * 100).round(decimals=2)
fig = px.bar(
    data_frame=df_percentual_motivo_afastamento.sort_values('percentual_motivo_afastamento', ascending=False), 
    x = 'questao_motivo_afastamento', 
    y = 'percentual_motivo_afastamento',
    color='questao_motivo_afastamento',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_motivo_afastamento': 'Percentual',
        'questao_motivo_afastamento': 'Motivo do afastamento'
    }
)
fig.update_layout(
    title='<b>Diferença percentual entre infectados por motivo do afastamento do trabalho</b>',
    yaxis_title='%',
    legend_title='Legenda',
    xaxis_visible=False,
    xaxis_showticklabels=False,
    width=1200, 
    height=600
)
fig.show()
df_percentual_motivo_afastamento.to_csv('dados/dados_streamlit/df_percentual_motivo_afastamento.csv', sep=',', index=False)

#### **<p style='color:gray'> Estatísticas decritivas dos métodos de testagem realizados</p>**

**Qual é a quantidade de testes aplicados por tipo de teste?**

In [56]:
df_qtd_tipo_teste_covid = spark.sql(
    '''
        SELECT tipo_teste, count(tipo_teste) AS qtd_tipo_teste
        FROM df_temp 
        WHERE tipo_teste != 'Não aplicável'
        GROUP BY tipo_teste
    '''
).toPandas()
fig = px.bar(
    data_frame=df_qtd_tipo_teste_covid.sort_values('qtd_tipo_teste', ascending=False), 
    x='tipo_teste',
    y='qtd_tipo_teste',
    color='tipo_teste',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_tipo_teste': 'Quantidade',
        'tipo_teste': 'Tipo de teste'
    }
)
fig.update_layout(
    title='<b>Quantidade de teste por tipo de teste</b>',
    legend_title='Legenda',
    width=600, 
    height=400
)
fig.update_xaxes(tickangle=45)
fig.show()
df_qtd_tipo_teste_covid.to_csv('dados/dados_streamlit/df_qtd_tipo_teste_covid.csv', sep=',', index=False)

In [57]:
df_qtd_tipo_teste = spark.sql(
    '''
        SELECT tipo_teste, count(tipo_teste) AS qtd_tipo_teste
        FROM df_temp 
        WHERE tipo_teste != 'Não aplicável'
        GROUP BY tipo_teste
    '''
).toPandas()
fig = px.pie(
    data_frame=df_qtd_tipo_teste_covid.sort_values('qtd_tipo_teste', ascending=False), 
    values='qtd_tipo_teste',
    names='tipo_teste',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_tipo_teste': 'Quantidade',
        'tipo_teste': 'Tipo de teste'
    }
)
fig.update_layout(
    title='<b>Diferença percentual de testes aplicados por tipo de teste</b>',
    legend_title='Legenda',
    width=600, 
    height=400
)
fig.update_xaxes(tickangle=45)
fig.show()
df_qtd_tipo_teste.to_csv('dados/dados_streamlit/df_qtd_tipo_teste.csv', sep=',', index=False)

**Qual é a quantidade de infectados por tipo de teste?**


In [58]:
df_qtd_testes_positivos_tipo_teste = spark.sql(
    '''
        SELECT tipo_teste, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY tipo_teste
    '''
).toPandas()
fig = px.bar(
    data_frame=df_qtd_testes_positivos_tipo_teste.sort_values('qtd_testes_positivos', ascending=False), 
    x='tipo_teste',
    y='qtd_testes_positivos',
    color='tipo_teste',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_validos': 'Quantidade',
        'tipo_teste': 'Tipo de teste'
    }
)
fig.update_layout(
    title='<b>Quantidade de testes positivos por tipo de teste</b>',
    legend_title='Legenda',
    yaxis_title='%',
    width=600, 
    height=400
)
fig.update_xaxes(tickangle=45)
fig.show()
df_qtd_testes_positivos_tipo_teste.to_csv('dados/dados_streamlit/df_qtd_testes_positivos_tipo_teste.csv', sep=',', index=False)

In [59]:
df_qtd_testes_validos_tipo_teste = spark.sql(
    '''
        SELECT tipo_teste, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY tipo_teste

    '''
).toPandas()
df_qtd_testes_positivos_tipo_teste = spark.sql(
    '''
        SELECT tipo_teste, count(teste_covid) AS qtd_testes_positivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY tipo_teste
    '''
).toPandas()
df_taxa_incidencia_tipo_teste = pd.merge(df_qtd_testes_validos_tipo_teste, df_qtd_testes_positivos_tipo_teste, on='tipo_teste')
df_taxa_incidencia_tipo_teste['taxa_incidencia_mil_habitantes'] = ((df_taxa_incidencia_tipo_teste['qtd_testes_positivos'] / df_taxa_incidencia_tipo_teste['qtd_testes_validos']) * 1000).round().astype(int)
fig = px.pie(
    data_frame=df_taxa_incidencia_tipo_teste.sort_values('taxa_incidencia_mil_habitantes', ascending=False), 
    values='taxa_incidencia_mil_habitantes',
    names='tipo_teste',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'taxa_incidencia_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'tipo_teste': 'Tipo de teste'
    }
)
fig.update_layout(
    title='<b>Diferença percentual da taxa de incidência por tipo de teste</b>',
    legend_title='Legenda',
    yaxis_title='%',
    width=600, 
    height=400
)
fig.show()
df_taxa_incidencia_tipo_teste.to_csv('dados/dados_streamlit/df_taxa_incidencia_tipo_teste.csv', sep=',', index=False)

**Qual é a quantidade de testes inconclusivos por tipo de teste?**

In [66]:
df_qtd_testes_inconclusivos_tipo_teste = spark.sql(
    '''
        SELECT tipo_teste, count(teste_covid) AS qtd_testes_inconclusivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Inconclusivo'
        GROUP BY tipo_teste

    '''
).toPandas()
fig = px.bar(
    data_frame=df_qtd_testes_inconclusivos_tipo_teste.sort_values('qtd_testes_inconclusivos', ascending=False), 
    x='tipo_teste',
    y='qtd_testes_inconclusivos',
    color='tipo_teste',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_validos': 'Quantidade',
        'tipo_teste': 'Tipo de teste'
    }
)
fig.update_layout(
    title='<b>Quantidade de testes inconclusivo</b>',
    legend_title='Legenda',
    width=600, 
    height=400
)
fig.show()
df_qtd_testes_inconclusivos_tipo_teste.to_csv('dados/dados_streamlit/df_qtd_testes_inconclusivos_tipo_teste.csv', sep=',', index=False)

In [ ]:
df_qtd_testes_validos_tipo_teste_inconclusivo = spark.sql(
    '''
        SELECT tipo_teste, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo' OR resultado_teste = 'Inconclusivo')
        GROUP BY tipo_teste

    '''
).toPandas()
df_qtd_testes_inconclusivos_tipo_teste_inconclusivo = spark.sql(
    '''
        SELECT tipo_teste, count(teste_covid) AS qtd_testes_inconclusivos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Inconclusivo'
        GROUP BY tipo_teste
    '''
).toPandas()
df_taxa_incidencia_tipo_teste_inconclusivo = pd.merge(df_qtd_testes_validos_tipo_teste_inconclusivo, df_qtd_testes_inconclusivos_tipo_teste_inconclusivo, on='tipo_teste')
df_taxa_incidencia_tipo_teste_inconclusivo['taxa_incidencia_mil_habitantes'] = ((df_taxa_incidencia_tipo_teste_inconclusivo['qtd_testes_inconclusivos'] / df_taxa_incidencia_tipo_teste_inconclusivo['qtd_testes_validos']) * 1000).round().astype(int)
fig = px.pie(
    data_frame=df_taxa_incidencia_tipo_teste_inconclusivo.sort_values('taxa_incidencia_mil_habitantes', ascending=False), 
    values='taxa_incidencia_mil_habitantes',
    names='tipo_teste',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'taxa_incidencia_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'tipo_teste': 'Tipo de teste'
    }
)
fig.update_layout(
    title='<b>Diferença percentual de testes com resultado inconclusivo por tipo de teste</b>',
    legend_title='Legenda',
    width=600, 
    height=400
)
fig.show()
df_taxa_incidencia_tipo_teste_inconclusivo.to_csv('dados/dados_streamlit/df_taxa_incidencia_tipo_teste_inconclusivo.csv', sep=',', index=False)

### Análise exploratória de características comportamentais

#### **<p style='color:gray'> Comportamento dos entrevistados em relação a busca por cuidados médicos</p>**

**Qual foi a porcentagem de sintomáticos que buscou atendimento médico?**

In [ ]:
df_sintomaticos_estabelecimento_saude = spark.sql(
    '''
        SELECT
            count(sintoma_covid) AS qtd_sintomaticos,
            CASE
            WHEN questao_estabelecimento_saude = 'Sim' THEN 'Buscou atendimento'
            WHEN questao_estabelecimento_saude = 'Não' THEN 'Não buscou atendimento'
            WHEN questao_estabelecimento_saude = 'Ignorado' THEN 'Ignorado'
            ELSE 'Não aplicável'
            END AS estabelecimento_saude
        FROM df_temp
        WHERE sintoma_covid = 'Sim' AND (questao_estabelecimento_saude != 'Não aplicável' AND questao_estabelecimento_saude != 'Ignorado')
        GROUP BY estabelecimento_saude
    '''
).toPandas()
fig = px.bar(
    data_frame=df_sintomaticos_estabelecimento_saude.sort_values('qtd_sintomaticos', ascending=False), 
    x='estabelecimento_saude', 
    y='qtd_sintomaticos',
    color='estabelecimento_saude',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_sintomaticos': 'Quantidade',
        'estabelecimento_saude': 'Comportamento'
    }
)
fig.update_layout(
    title='<b>Quantidade de sintomáticos que buscaram atendimento médico</b>',
    showlegend=False,
    width=600,
    height=400
)
fig.show()
df_sintomaticos_estabelecimento_saude.to_csv('dados/dados_streamlit/df_sintomaticos_estabelecimento_saude.csv', sep=',', index=False)

In [ ]:
df_sintomaticos_estabelecimento_saude = spark.sql(
    '''
        SELECT
            count(sintoma_covid) AS qtd_sintomaticos,
            CASE
            WHEN questao_estabelecimento_saude = 'Sim' THEN 'Buscou atendimento'
            WHEN questao_estabelecimento_saude = 'Não' THEN 'Não buscou atendimento'
            WHEN questao_estabelecimento_saude = 'Ignorado' THEN 'Ignorado'
            ELSE 'Não aplicável'
            END AS estabelecimento_saude
        FROM df_temp
        WHERE sintoma_covid = 'Sim' AND (questao_estabelecimento_saude != 'Não aplicável' AND questao_estabelecimento_saude != 'Ignorado')
        GROUP BY estabelecimento_saude
    '''
).toPandas()
fig = px.pie(
    data_frame=df_sintomaticos_estabelecimento_saude.sort_values('qtd_sintomaticos', ascending=False), 
    values='qtd_sintomaticos', 
    names='estabelecimento_saude',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_sintomaticos': 'Quantidade',
        'estabelecimento_saude': 'Comportamento'
    }
)
fig.update_layout(
    title='<b>Percentual de sintomáticos que buscaram atendimento médico</b>',
    width=600,
    height=400
)
fig.show()
df_sintomaticos_estabelecimento_saude.to_csv('dados/dados_streamlit/df_sintomaticos_estabelecimento_saude.csv', sep=',', index=False)

#### **<p style='color:gray'> Comportamento dos entrevistados em relação a medidas de isolamento social</p>**

**Qual é a quantidade de pacientes sintomáticos que efetuou medida de isolamento social?**

In [ ]:
df_sintomaticos_permaneceu_casa = spark.sql(
    '''
        SELECT questao_permaneceu_casa, count(sintoma_covid) as qtd_sintomaticos
        FROM df_temp 
        WHERE sintoma_covid = 'Sim' AND (questao_permaneceu_casa = 'Sim' OR questao_permaneceu_casa = 'Não')
        GROUP BY questao_permaneceu_casa
    '''
).toPandas()
fig = px.bar(
    data_frame=df_sintomaticos_permaneceu_casa.sort_values('qtd_sintomaticos', ascending=False), 
    x='questao_permaneceu_casa', 
    y='qtd_sintomaticos',
    color='questao_permaneceu_casa',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_sintomaticos': 'Quantidade',
        'questao_permaneceu_casa': 'Isolamento'
    }
)
fig.update_layout(
    title='<b>Quantidade de pacientes sintomáticos que adotaram medida de isolamento social</b>',
    showlegend=False,
    width=600, 
    height=400
)
fig.show()
df_sintomaticos_permaneceu_casa.to_csv('dados/dados_streamlit/df_sintomaticos_permaneceu_casa.csv', sep=',', index=False)

In [ ]:
df_sintomaticos_permaneceu_casa = spark.sql(
    '''
        SELECT questao_permaneceu_casa, count(sintoma_covid) as qtd_sintomaticos
        FROM df_temp 
        WHERE sintoma_covid = 'Sim' AND (questao_permaneceu_casa = 'Sim' OR questao_permaneceu_casa = 'Não')
        GROUP BY questao_permaneceu_casa
    '''
).toPandas()
fig = px.pie(
    data_frame=df_sintomaticos_permaneceu_casa, 
    values = 'qtd_sintomaticos', 
    names = 'questao_permaneceu_casa',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_sintomaticos': 'Quantidade',
        'questao_permaneceu_casa': 'Isolamento'
    }
)
fig.update_layout(
    title='<b>Percentual de pacientes sintomáticos que adotaram medida de isolamento social</b>',
    legend_title='Legenda',
    width=600, 
    height=400
)
fig.show()
df_sintomaticos_permaneceu_casa.to_csv('dados/dados_streamlit/df_sintomaticos_permaneceu_casa.csv', sep=',', index=False)

**Qual é a percentual de pacientes sintomáticos que efetuou medida de isolamento social por região?**

In [ ]:
df_sintomaticos_permaneceu_casa_regiao = spark.sql(
    '''
        SELECT regiao, questao_permaneceu_casa, count(sintoma_covid) as qtd_sintomaticos
        FROM df_temp 
        WHERE sintoma_covid = 'Sim' AND questao_permaneceu_casa = 'Sim'
        GROUP BY regiao, questao_permaneceu_casa
    '''
).toPandas()
fig = px.pie(
    data_frame=df_sintomaticos_permaneceu_casa_regiao, 
    values = 'qtd_sintomaticos', 
    names = 'regiao',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_sintomaticos': 'Quantidade',
        'regiao': 'Região'
    }
)
fig.update_layout(
    title='<b>Porcentagem de pacientes sintomáticos que adotaram medida de isolamento social por região</b>',
    legend_title='Legenda',
    width=600, 
    height=400
)
fig.show()
df_sintomaticos_permaneceu_casa_regiao.to_csv('dados/dados_streamlit/df_sintomaticos_permaneceu_casa_regiao.csv', sep=',', index=False)

**Qual é o percentual de pacientes sintomáticos que efetuou medida de isolamento social por estado?**

In [ ]:
df_sintomaticos_permaneceu_casa_estado = spark.sql(
    '''
        SELECT uf, questao_permaneceu_casa, count(sintoma_covid) as qtd_sintomaticos
        FROM df_temp 
        WHERE sintoma_covid = 'Sim' AND questao_permaneceu_casa = 'Sim'
        GROUP BY uf, questao_permaneceu_casa
    '''
).toPandas()
fig = px.bar(
    data_frame=df_sintomaticos_permaneceu_casa_estado.sort_values('qtd_sintomaticos', ascending=False), 
    x='uf',
    y='qtd_sintomaticos',
    color='uf',
    color_discrete_sequence=['#67000D'],
    labels={
        'qtd_sintomaticos': 'Quantidade',
        'uf': 'Estado'
    }
)
fig.update_layout(
    title='<b>Percentual de pacientes sintomáticos que adotaram medida de isolamento social por estado</b>',
    yaxis_title='%',
    showlegend=False,
    width=1000, 
    height=600
)
fig.update_xaxes(tickangle=45)
fig.show()
df_sintomaticos_permaneceu_casa_estado.to_csv('dados/dados_streamlit/df_sintomaticos_permaneceu_casa_estado.csv', sep=',', index=False)

**Qual é o percentual de infectados que se automedicaram?**

In [ ]:
df_testes_positivos_automedicacao = spark.sql(
    '''
        SELECT  
            questao_remedio_conta_propria AS resposta,
            count(questao_remedio_conta_propria) AS qtd_automedicacao
        FROM df_temp
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND questao_remedio_conta_propria = 'Sim' AND questao_remedio_orientacao_medica = 'Não'
        GROUP BY resposta
    '''
).toPandas()
df_testes_positivos_medicacao_orientada = spark.sql(
    '''
        SELECT  
            questao_remedio_orientacao_medica AS resposta,
            count(questao_remedio_orientacao_medica) AS qtd_medicacao_orientada
        FROM df_temp
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND questao_remedio_orientacao_medica = 'Sim' AND questao_remedio_conta_propria = 'Não'
        GROUP BY resposta
    '''
).toPandas()
df_testes_positivos_medicacao_ambos = spark.sql(
    '''
        SELECT  
            questao_remedio_orientacao_medica AS resposta,
            count(questao_remedio_orientacao_medica) AS questao_remedio_ambos
        FROM df_temp
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND questao_remedio_orientacao_medica = 'Sim' AND questao_remedio_conta_propria = 'Sim'
        GROUP BY resposta
    '''
).toPandas()
df_testes_positivos_medicacao = df_testes_positivos_automedicacao.merge(df_testes_positivos_medicacao_orientada, on='resposta')
df_testes_positivos_medicacao = df_testes_positivos_medicacao.merge(df_testes_positivos_medicacao_ambos, on='resposta')
fig = go.Figure(data=[
    go.Bar(name='Automedicados', x=df_testes_positivos_medicacao.resposta, y=df_testes_positivos_medicacao.qtd_automedicacao),
    go.Bar(name='Medicados com orientação médica', x=df_testes_positivos_medicacao.resposta, y=df_testes_positivos_medicacao.qtd_medicacao_orientada),
    go.Bar(name='Ambos', x=df_testes_positivos_medicacao.resposta, y=df_testes_positivos_medicacao.questao_remedio_ambos)   
    ]
)
fig.update_layout(
    title='<b>Quantidade de infectados automedicados e medicados com orientacao médica</b>',
    legend_title='Legenda',
    yaxis_title='Quantidade',
    xaxis_visible=False,
    width=800,
    height=600
)
fig.data[0].marker.color = ['#67000d',] * 2
fig.data[1].marker.color = ['#A50F15',] * 2 
fig.data[2].marker.color = ['#FB6A4A',] * 2 
fig.show()
df_testes_positivos_medicacao.to_csv('dados/dados_streamlit/df_testes_positivos_medicacao.csv', sep=',', index=False)

### Análise exploratória de características clínicas

#### **<p style='color:gray'> Estatísticas descritivas de pacientes sintomáticos e assintomáticos</p>**

**Qual é o percentual de infectados sintomáticos e assintomáticos?**

Observa-se que ampla maioria de infectados são assintomáticos.

In [ ]:
df_qtd_testes_positivos_sintomaticos = spark.sql(
    '''
        SELECT 
            CASE
            WHEN sintoma_covid = 'Sim' THEN 'Sintomáticos'
            ELSE 'Assintomáticos' 
            END AS sintoma_covid,
            count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY sintoma_covid
    '''
).toPandas()
fig = px.pie(
    data_frame=df_qtd_testes_positivos_sintomaticos.sort_values('qtd_testes_positivos', ascending=False),
    values = 'qtd_testes_positivos',
    names = 'sintoma_covid',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos':'Quantidade',
        'sintoma_covid':'Condição'
    }
)
fig.update_traces(
    pull=[0.1, 0]

)
fig.update_layout(
    title='<b>Percentual de infectados sintomáticos e assintomáticos</b>',
    legend_title='Legenda',
    width=800,
    height=600
)
fig.show()
df_qtd_testes_positivos_sintomaticos.to_csv('dados/dados_streamlit/df_qtd_testes_positivos_sintomaticos.csv', sep=',', index=False)

**Qual é o percentual de infectados por fator de risco?**

In [ ]:
df_percentual_testes_positivos_fator_risco = spark.sql(
'''
    SELECT
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Diabetes%' THEN 1 ELSE 0 END) /sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100 AS Diabetes,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Hipertensao%' THEN 1 ELSE 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100  AS Hipertensao,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Doenca respiratoria%' THEN 1 else 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100 as Doenca_respiratoria,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Idoso%' THEN 1 ELSE 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100  AS Idoso,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Cancer%' THEN 1 ELSE 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100  as Cancer,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Doenca cardiaca%' THEN 1 ELSE 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100  AS Doenca_cardiaca
    FROM df_temp
    WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
'''
).toPandas()
df_percentual_testes_positivos_fator_risco = df_percentual_testes_positivos_fator_risco.T.reset_index().rename(columns={0: 'percentual_testes_positivos', 'index': 'fator_risco_covid'})
df_percentual_testes_positivos_fator_risco['fator_risco_covid'] = [i.replace('_', ' ') for i in df_percentual_testes_positivos_fator_risco.fator_risco_covid.values]
df_percentual_testes_positivos_fator_risco.sort_values('percentual_testes_positivos', ascending=False, inplace=True)
fig = px.bar(
    data_frame = df_percentual_testes_positivos_fator_risco,
    x = 'fator_risco_covid',
    y = 'percentual_testes_positivos',
    color= 'fator_risco_covid',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_testes_positivos': 'Percentual de testes positivos',
        'fator_risco_covid': 'Fator de risco'
    }
)
fig.update_layout(
    title='<b>Percentual de infectados por fator de risco</b>',
    showlegend=False,
    yaxis_title='%',
    width=800,
    height=600
)
fig.update_xaxes(tickangle=45)
fig.show()
df_percentual_testes_positivos_fator_risco.to_csv('dados/dados_streamlit/df_percentual_testes_positivos_fator_risco.csv', sep=',', index=False)

**Qual é porcentagem de infectados por sintoma?**

In [ ]:
df_percentual_testes_positivos_tipo_sintoma = spark.sql(
    '''
    SELECT
          (sum(CASE WHEN descricao_sintoma_covid LIKE '%Febre%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END)) * 100 AS Febre,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Tosse%'THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Tosse,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Dor de garganta%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Dor_de_garganta,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Dificuldade para respirar%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Dificuldade_de_respirar,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Dor de cabeça%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Dor_de_cabeca,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Dor no peito%'THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Dor_no_peito,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Nausea%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Nausea,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Fadiga%'THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Fadiga,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Dor nos olhos%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Dor_nos_olhos,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Perda de olfato ou paladar%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Perda_de_olfato_ou_paladar,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Dor muscular%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Dor_muscular,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Diarreia%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Diarreia
    FROM df_temp
    WHERE resultado_teste = 'Positivo'
    '''
).toPandas()
df_percentual_testes_positivos_tipo_sintoma = df_percentual_testes_positivos_tipo_sintoma.T.reset_index().rename(columns={0: 'percentual_testes_positivos', 'index':'sintoma'})
df_percentual_testes_positivos_tipo_sintoma['sintoma'] = [i.replace('_', ' ') for i in df_percentual_testes_positivos_tipo_sintoma.sintoma.values]
df_percentual_testes_positivos_tipo_sintoma.sort_values('percentual_testes_positivos', ascending=False, inplace=True)
fig = px.bar(
    data_frame = df_percentual_testes_positivos_tipo_sintoma,
    x = 'sintoma',
    y = 'percentual_testes_positivos',
    color= 'sintoma',
    color_discrete_sequence=['#67000D'],
    labels={
        'percentual_testes_positivos': 'Percentual de testes positivos',
        'sintoma': 'Sintoma'
    }
)
fig.update_layout(
    title='<b>Percentual de infectados por tipo de sintoma</b>',
    showlegend=False,
    yaxis_title='%',
    width=800,
    height=600
)
fig.update_xaxes(tickangle=45)
fig.show()
df_percentual_testes_positivos_tipo_sintoma.to_csv('dados/dados_streamlit/df_percentual_testes_positivos_tipo_sintoma.csv', sep=',', index=False)

**Qual é a percentual de infectados que tiveram internação?**

In [ ]:
df_qtd_testes_positivos_internacao = spark.sql(

    '''
        SELECT
            count(teste_covid) AS qtd_testes_positivos,
            CASE
            WHEN questao_internacao = 'Sim' THEN 'Internado'
            ELSE 'Não internado' 
            END AS questao_internacao
        FROM df_temp
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo'
        GROUP BY questao_internacao
    '''
).toPandas()
fig = px.pie(
    data_frame=df_qtd_testes_positivos_internacao.sort_values('qtd_testes_positivos', ascending=False),
    values='qtd_testes_positivos',
    names='questao_internacao',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos':'Quantidade',
        'questao_internacao':'Condição'
    }
)
fig.update_traces(
    pull=[0.3, 0],
    texttemplate='%{percent:.2%}'
)
fig.update_layout(
    title='<b>Percentual de infectados internados</b>',
    legend_title='Legenda',
    width=800,
    height=600
)
fig.show()
df_qtd_testes_positivos_internacao.to_csv('dados/dados_streamlit/df_qtd_testes_positivos_internacao.csv', sep=',', index=False)

**Qual é a porcentagem de internados que precisaram de respiração artificial?**

In [ ]:
df_testes_positivos_respiracao_artificial = spark.sql( 
    '''
        SELECT
            count(teste_covid) AS qtd_testes_positivos,
            CASE
            WHEN questao_internacao_ajuda_respirar = 'Sim' THEN 'Respiração artificial'
            WHEN questao_internacao_ajuda_respirar = 'Não' THEN 'Respiração natural'
            WHEN questao_internacao_ajuda_respirar = 'Ignorado' THEN 'Respiração natural'
            ELSE 'Não aplicável'
            END AS questao_internacao_ajuda_respirar
        FROM df_temp
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' AND questao_internacao_ajuda_respirar != 'Não aplicável'
        GROUP BY questao_internacao_ajuda_respirar
    '''
).toPandas()
df_testes_positivos_respiracao_artificial.groupby('questao_internacao_ajuda_respirar').sum().reset_index(inplace=True)
fig = px.pie(
    data_frame=df_testes_positivos_respiracao_artificial.sort_values('qtd_testes_positivos', ascending=False),
    values='qtd_testes_positivos',
    names='questao_internacao_ajuda_respirar',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos':'Quantidade',
        'questao_internacao_ajuda_respirar':'Condição'
    }
)
fig.update_traces(
    texttemplate='%{percent:.2%}'
)
fig.update_layout(
    title='<b>Percentual de infectados internados com respiração articial</b>',
    legend_title='Legenda',
    width=600,
    height=400
)
fig.show()
df_testes_positivos_respiracao_artificial.to_csv('dados/dados_streamlit/df_testes_positivos_respiracao_artificial.csv', sep=',', index=False)

**a**

In [65]:
df_qtd_idade = spark.sql(
    '''
        SELECT 
            CASE 
            WHEN idade BETWEEN  0 AND 9 THEN '0 - 10 anos'
            WHEN idade BETWEEN 10 AND 19 THEN '10 - 19 anos'
            WHEN idade BETWEEN 20 AND 29 THEN '20 - 29 anos'
            WHEN idade BETWEEN 30 AND 39 THEN '30 - 39 anos'
            WHEN idade BETWEEN 40 AND 49 THEN '40 - 49 anos'
            WHEN idade BETWEEN 50 AND 59 THEN '50 - 59 anos'
            WHEN idade >= 60 THEN 'Maior que 60 anos' 
            END AS idade,
            count(idade) as qtd_idade
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND resultado_teste = 'Positivo' 
        GROUP BY idade
    '''
).toPandas()
df_qtd_idade.sort_values('qtd_idade', ascending=False, inplace=True)
fig = px.pie(
    data_frame=df_qtd_idade.sort_values('qtd_idade', ascending=False),
    values='qtd_idade',
    names='idade',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_idade':'Quantidade',
        'idade':'Faixa etária'
    }
)
fig.update_traces(
    texttemplate='%{percent:.2%}'
)
fig.update_layout(
    title='<b>Percentual de entrevistados por faixa etária</b>',
    legend_title='Legenda',
    width=600,
    height=400
)
fig.show()
df_qtd_idade.to_csv('dados/dados_streamlit/df_qtd_idade.csv', sep=',', index=False)